In [2]:
install.packages("udpipe")
install.packages("topicmodels")
install.packages("tm")
install.packages("slam")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependency ‘modeltools’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [4]:
load("data/mymovies.Rdata")

In [5]:
library(topicmodels)
library(udpipe)
library(tm) 
library(slam)
txt <- tolower(mymovies$tomatoConsensus)
txt <- removeWords(txt, words = stopwords("english"))
mycorpus <- Corpus(x = VectorSource(txt))
mycorpus <- tm_map(mycorpus, FUN=function(x) stripWhitespace(x))
## Build Document Term matrix
mydtm <- DocumentTermMatrix(mycorpus, control = list(wordLengths = c(2, Inf), bounds = list(global = c(5, Inf)))) ## tfidf
term_tfidf <- tapply(mydtm$v/row_sums(mydtm)[mydtm$i], mydtm$j, mean) * log2(nrow(mydtm)/col_sums(mydtm > 0))
mydtm <- mydtm[, term_tfidf >= quantile(term_tfidf, 0.5)]
## Work only on the top 500 words if more words are occuring
termfreq <- col_sums(mydtm)
termfreq <- sort(termfreq, decreasing = TRUE)
termfreq <- head(termfreq, 500)
mydtm <- mydtm[, intersect(colnames(mydtm), names(termfreq))]
mydtm <- mydtm[, col_sums(mydtm) > 5]
mydtm <- mydtm[row_sums(mydtm) > 0, ]
dim(mydtm)

[1] 2114  500

In [6]:
set.seed(123456789)
m <- LDA(x = mydtm, k = 7, method = "VEM", control = list(alpha = 0.1))
m

A LDA_VEM topic model with 7 topics.

In [7]:
slotNames(m)

[1] "alpha"           "call"            "Dim"             "control"        
 [5] "k"               "terms"           "documents"       "beta"           
 [9] "gamma"           "wordassignments" "loglikelihood"   "iter"           
[13] "logLiks"         "n"

In [8]:
# posterior probability
x <- posterior(m, newdata = mydtm)$topics
x[1:2, ]

,1,2,3,4,5,6,7
1,0.03707316,0.03707317,0.03707316,0.77756101,0.03707316,0.03707316,0.03707316
2,0.02130044,0.02130044,0.02130044,0.02130044,0.02130045,0.02130044,0.87219733


In [10]:
# words emitted per topic
x <- posterior(m, newdata = mydtm)$terms
apply(x, MARGIN=1, FUN=function(x, top = 5){
    idx <- order(x, decreasing = TRUE)
    out <- data.frame(term = names(x)[idx], prob = x[idx])
    head(out, n = top)
})

,term,prob
romance,romance,0.02523687
less,less,0.02458398
beautiful,beautiful,0.02258784
gross,gross,0.02020333
lives,lives,0.01883576
,term,prob
bad,bad,0.04269775
moments,moments,0.03273494
children,children,0.02988843
silly,silly,0.02924999


In [11]:
logLik(m)

'log Lik.' -31517.57 (df=3501)

In [12]:
topicmodels::perplexity(m)

[1] 458.6313

In [13]:
terms(m, 5)

Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7
romance,bad,derivative,audience,scares,sequel,eye
less,moments,audiences,heartfelt,road,unfortunately,complex
beautiful,children,scary,creepy,appealing,away,exciting
gross,silly,gory,spirit,remake,heavy,nothing
lives,muddled,engrossing,feature,suspense,comedies,commentary


In [14]:
install.packages("LDAvis")
install.packages("servr")
library(LDAvis)
library(servr) 

also installing the dependencies ‘proxy’, ‘RJSONIO’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [16]:
json <- createJSON(phi = posterior(m)$terms, 
                   theta = posterior(m)$topics, 
                   doc.length = row_sums(mydtm), 
                   vocab = colnames(mydtm), 
                   term.frequency = col_sums(mydtm))
serVis(json)

In [ ]:
# exercise 1: Find + visualise topics in the movie plots.
# Make an appropriate selection of POS tags (nouns, verbs) to make the topics 

In [ ]:
# exercise 2:  Find + visualise topics in the airbnb customer feedback.
# Make an appropriate selection of POS tags (nouns, verbs) to make the topics